# Audiobooks business case

### Import the relevant libraries

In [24]:
import numpy as np
import tensorflow as tf

### Data extraction

In [25]:
# let's create a temporary variable npz, where the three Audiobooks datasets will be stored.
npz = np.load('Audiobooks_data_train.npz')

# Extract the inputs, and change them to float type.
train_inputs = npz['inputs'].astype(np.float)
# Targets must be int because of the sparse_categorical_crossentropy loss function, so they can be easily one-hot encoded
train_targets = npz['targets'].astype(np.int)

# Load the validation data in the temporary variable and extract intputs and targets
npz = np.load('Audiobooks_data_validation.npz')
validation_inputs, validation_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

# Same with the test data
npz = np.load('Audiobooks_data_test.npz')
test_inputs, test_targets = npz['inputs'].astype(np.float), npz['targets'].astype(np.int)

C:\Users\Usuario\AppData\Local\Temp/ipykernel_6496/2403072159.py:5: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  train_inputs = npz['inputs'].astype(np.float)
C:\Users\Usuario\AppData\Local\Temp/ipykernel_6496/2403072159.py:7: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.o

### Model creation

In [36]:
# Set the input and output sizes
input_size = 10
output_size = 2
# Use same hidden layer size for both hidden layers.
hidden_layer_size = 50
    
model = tf.keras.Sequential([
    tf.keras.layers.Dense(hidden_layer_size, activation='relu'), # 1st hidden layer
    tf.keras.layers.Dense(hidden_layer_size, activation='tanh'), # 2nd hidden layer
    tf.keras.layers.Dense(output_size, activation='softmax') # output layer
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

batch_size = 50
max_epochs = 100

early_stopping = tf.keras.callbacks.EarlyStopping(patience=2)

model.fit(train_inputs, 
          train_targets, 
          batch_size=batch_size,
          epochs=max_epochs, # Epochs that we will train for (assuming there is not early stopping)
          # Callbacks are functions called when a task is completed
          # In this case, the task is to check if val_loss is increasing
          callbacks=[early_stopping],
          validation_data=(validation_inputs, validation_targets), #I could also have splitted the data into to npz (train and test) 
          # and then make the split with validation_split
          verbose = 1
          )  

Epoch 1/100
72/72 [==============================] - 1s 7ms/step - loss: 0.5768 - accuracy: 0.6689 - val_loss: 0.4286 - val_accuracy: 0.7562
Epoch 2/100
72/72 [==============================] - 0s 3ms/step - loss: 0.4214 - accuracy: 0.7647 - val_loss: 0.3777 - val_accuracy: 0.7942
Epoch 3/100
72/72 [==============================] - 0s 3ms/step - loss: 0.3873 - accuracy: 0.7936 - val_loss: 0.3578 - val_accuracy: 0.8031
Epoch 4/100
72/72 [==============================] - 0s 3ms/step - loss: 0.3670 - accuracy: 0.8002 - val_loss: 0.3467 - val_accuracy: 0.8143
Epoch 5/100
72/72 [==============================] - 0s 3ms/step - loss: 0.3585 - accuracy: 0.8041 - val_loss: 0.3453 - val_accuracy: 0.8009
Epoch 6/100
72/72 [==============================] - 0s 3ms/step - loss: 0.3548 - accuracy: 0.7998 - val_loss: 0.3348 - val_accuracy: 0.8322
Epoch 7/100
72/72 [==============================] - 0s 3ms/step - loss: 0.3519 - accuracy: 0.8162 - val_loss: 0.3273 - val_accuracy: 0.8322
Epoch 8/100
7

## Testing the model

After training on the training data and validating on the validation data, we test the final prediction power of our model by running it on the test dataset that the algorithm has NEVER seen before.

In [10]:
test_loss, test_accuracy = model.evaluate(test_inputs, test_targets)

14/14 [==============================] - 0s 2ms/step - loss: 0.3580 - accuracy: 0.8214


In [11]:
print('\nTest loss: {0:.2f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))


Test loss: 0.36. Test accuracy: 82.14%
